In [1]:
%reload_ext autoreload
%autoreload 2

# makes all experimental recording ready for analysis
import pandas as pd
import numpy as np
import main_funcs as mfun
from pathlib import Path
import os
import glob
import paq2py as paq_reader
import pickle
import utils_funcs as utils


# Get the list of recordings
info = mfun.analysis()
#info.recordingList.head()

Env: clapfcstimulation
Computer: Huriye Windows


In [2]:
# Check suite2p extraction
# run run_suite2p.ipynb in suite2p environment 
# Extraction has already done with another code in suite2p environment. 
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    if info.recordingList.calciumImaging[ind] ==1: # if imaging recording is done
        #check if exist
        filepathname = (info.recordingList.filepathname[ind] +'_t-' + 
                        format(int(info.recordingList.recordingID[ind]), '03d') +
                        '\\suite2p\\')
        #update table accordingly
        try:
            if os.path.isdir(filepathname):
                filepath = Path(filepathname,'plane0','F.npy')
                if filepath.is_file(): info.recordingList.loc[ind,'suite2pextraction']=1
            else:
                info.recordingList.loc[ind,'suite2pextraction']=0
        except:
            print('Suite2p folder is created without imaging trace:', + ind)
            info.recordingList.loc[ind,'suite2pextraction']=0

# display the output
print( "Imaging Data Exist: " + 
      str(info.recordingList.calciumImaging.sum()) +"/" + str(info.recordingList.shape[0]))
print( "Suite2P Extraction Completed: " + 
      str(info.recordingList.suite2pextraction.sum()) +"/" + str(info.recordingList.calciumImaging.sum()))
notExtractedFiles = info.recordingList[info.recordingList['suite2pextraction'] != 1]
if notExtractedFiles.shape[0]!=0: notExtractedFiles.head()

Imaging Data Exist: 280/519
Suite2P Extraction Completed: 280.0/280


In [3]:
# Check pupil extraction - STEP 1: Stack to video conversion
# enviroment: DEEPLABCUT (specific requirements in pupil extraction. Hence specific env was used.)
# Pupil data saved as stacktiffs - convert them to avi for DeepLabCut Analysis
#import main_functions as mfun
#import glob
#info = mfun.analysis()

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    if info.recordingList.pupilImaging[ind] ==1: # if pupil recording is done
        #check if exist
        filepathname = (info.recordingList.filepathname[ind] +'_p-' + 
                        format(int(info.recordingList.recordingID[ind]), '03d') )
        filenameAVI  = [f for f in glob.glob(filepathname + "\\*.avi")]
        filenameTIF  = [f for f in glob.glob(filepathname + "\\*.tif")]
        # convert files if not converted already
        if len(filenameAVI)==1:
            info.recordingList.loc[ind,'tiffconverted']=1
        elif len(filenameAVI)!=1 & len(filenameTIF)==1:
            info.recordingList.loc[ind,'tiffconverted']=0
            filename = filenameTIF[0]
            print('Coverting:' + filename)
            outputsavename =  filenameTIF[0][:-4] + '.avi' #'D:\\pupil' + filenameTIF[0][41:-4] +'.avi' #
            print(outputsavename)
            mfun.convert_tiff2avi (filename,outputsavename)
        else:
            info.recordingList.loc[ind,'tiffconverted']=0
            print('No pupil data found :', filepathname)

# display the output
print( "Stack to video conversion Completed: " + 
      str(info.recordingList.tiffconverted.sum()) +"/" + str(info.recordingList.pupilImaging.sum()))
notExtractedFiles = info.recordingList[info.recordingList['tiffconverted'] != 1]
if notExtractedFiles.shape[0]!=0: notExtractedFiles.head()

Stack to video conversion Completed: 322.0/322


In [4]:
# Check pupil extraction - STEP 2: Video to CSV file conversion
# 'run_DLCforpupil.py' needs to be run in a different enviroment: DEEPLABCUT (specific requirements in pupil extraction. Hence specific env was used.)
#### this function save extracted values from avi videos to csv
# Here, we only look for output csv files if created.

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    if info.recordingList.tiffconverted[ind] ==1: # if pupil recording is done & converted to .avi file
        #check if exist
        filepathname = (info.recordingList.filepathname[ind] +'_p-' + 
                        format(int(info.recordingList.recordingID[ind]), '03d') )
        filenameCSV  = [f for f in glob.glob(filepathname + "\\*.csv")]

        # calculate pupil changes if not calculatated already
        try:
            if len(filenameCSV)==1:
                info.recordingList.loc[ind,'pupilcsvcreated']=1
            elif len(filenameAVI)!=1 & len(filenameCSV)==1:
                info.recordingList.loc[ind,'pupilcsvcreated']=0
                filename = filenameCSV[0]
                print('Coverting:' + filename)
                outputsavename = filenameCSV[0][:-4] + '.csv'
                mfun.convert_tiff2avi (filename,outputsavename)
        except:
            print('Pupil size extraction problem :', + filepathname)

# display the output
print( "Pupil size extraction Completed: " + 
      str(info.recordingList.pupilcsvcreated.sum()) +"/" + str(info.recordingList.pupilImaging.sum()))
notExtractedFiles = info.recordingList[info.recordingList['pupilcsvcreated'] != 1]
if notExtractedFiles.shape[0]==0: print('All CSV files for pupil data are created')

Pupil size extraction Completed: 322.0/322


In [5]:
# Check if PAQ data is extracted
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    try:
        #print(str(ind)+ ' Creating: ' + info.recordingList.analysispathname[ind])

        filepathname = (info.recordingList.filepathname[ind] +'_t-' + 
                                    format(int(info.recordingList.recordingID[ind]), '03d') )
        #filenameTIF  = [f for f in glob.glob(filepathname + "\\*.tif")] # datapath
        filenamePAQ  = [f for f in glob.glob(info.recordingList.filepathname[ind]  + # paqs_loc
                                            '*'+ format(int(info.recordingList.recordingID[ind]), '03d') + '.paq')]
        
        savepathname = info.recordingList.analysispathname[ind]
        if not os.path.exists(savepathname): os.makedirs(savepathname)
        filenamePAQ_analysis = [f for f in glob.glob(savepathname + 'paq-data.pkl')] # paq analysis file
        filenamePAQ_analysis_training = [f for f in glob.glob(savepathname + 'training-paq-data.pkl')] # paq analysis file

        if (len(filenamePAQ_analysis)==1) | (len(filenamePAQ_analysis_training)==1) :
            info.recordingList.loc[ind,'paqExtraction'] = 1
            
        elif  (len(filenamePAQ_analysis)==0) :
            print('Extracting PAQ data: ' + filenamePAQ[0])
            paq_data = paq_reader.paq_read( file_path=filenamePAQ[0], plot=True, save_path=savepathname) # we need this below
            #optoStimTimes = utils.stim_start_frame (paq=paq_data, stim_chan_name='optoLoopback')
            #if (len(optoStimTimes)!=40) and (len(optoStimTimes)!=20): # Two different recording types
            #    print('Opto stim times are not 40: ' +str(len(optoStimTimes)))
            #    info.recordingList.loc[ind,'paqExtraction'] = 0
            if filenamePAQ[0].find('trainingDay')<0:
                filenamePAQ_analysis = savepathname + 'paq-data.pkl'
            else:
                filenamePAQ_analysis = savepathname + 'training-paq-data.pkl'
            
            with open(filenamePAQ_analysis, 'wb') as f:
                pickle.dump(paq_data, f)
            info.recordingList.loc[ind,'paqExtraction'] = 1
        else:
            print('Something weird about this recording: ' + savepathname)
            info.recordingList.loc[ind,'paqExtraction'] = 0
    except:
        print('Failed to extract PAQ data: ' + str(ind))
        info.recordingList.loc[ind,'paqExtraction'] = 0

# display the output
print( "PAQ extraction Completed: " + 
      str(info.recordingList.paqExtraction.sum()) +"/" + str(info.recordingList.shape[0]))
notExtractedFiles = info.recordingList[info.recordingList['paqExtraction'] != 1]
if notExtractedFiles.shape[0]==0: print('All paq data are extracted')

PAQ extraction Completed: 519.0/519
All paq data are extracted


In [ ]:
  ###  TO PLOT each behaviour sessions  
import matplotlib.pyplot as plt
import plot_funcs as pfun

fRate = 20000
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    savepathname = info.recordingList.analysispathname[ind]
    pathname = [f for f in glob.glob(savepathname + 'training-paq-data.pkl')]
   # try:
    if  4>3:
        if (len(pathname)==1)& (ind>419):
            paqData = pd.read_pickle (pathname[0])
            # Get the stim start times 
            trialStartTimes = utils.paq_data (paqData, 'maskerLED', 1, threshold_ttl=True)

            trialStartTimes = trialStartTimes - (2*fRate)
            
            licks, trial_licks = utils.lick_binner(savepathname, trialStartTimes,'lickDetection', stimulation = False)
            waterPoints, trial_water = utils.lick_binner(savepathname,trialStartTimes, 'waterDelivery', stimulation = False)

            if len(trial_water)<500:
                fig = plt.figure(str(ind))
                for i, array in enumerate(trial_licks):
                    plt.plot(array, np.ones_like(array)+i, 'bo',markersize = 1)
                    plt.plot(trial_water[i], np.ones_like(trial_water[i])+i, 'rx',markersize = 2)
                
                plt.xlim(0, 8*fRate)
                plt.xticks (range(0,(8*fRate)+1,fRate), range(-2,7,1))
                plt.ylabel('Trials')
                plt.xlabel('Time (sec)')
                if ind>331:
                    fname = savepathname[-20:-1]
                else:
                    fname = savepathname[-21:-1]
                plt.title(fname +' '  + str(len(trialStartTimes)))
                pfun.save_figure( 'Behaviour_session',savepathname)
                savepathname = 'C:\\Users\\Huriye\Documents\\code\\clapfcstimulation\\figs\\beh_session\\'
                pfun.save_figure( fname,savepathname)
                plt.close(fig)
            else: 
                print(str(ind) +'Problem: '+ str(len(trial_water))+' Path:' + pathname[0] )
    #except:
      #  print('Failed to plot: ' + str(ind))
        
    

In [6]:
# Check if calcium imaging data is extracted from suite 2p file
for ind, recordingDate in enumerate(info.recordingList.recordingDate):  
    #try:
        # define paths to retrieve data from disc
        filepathname = (info.recordingList.filepathname[ind] +'_t-' + 
                                format(int(info.recordingList.recordingID[ind]), '03d') )
        filenameTIF  = [f for f in glob.glob(filepathname + "\\*.tif")] # datapath
        savepathname = info.recordingList.analysispathname[ind]
        filenameTIF_analysis = [f for f in glob.glob(savepathname + 'imaging-data.pkl')]
        if (info.recordingList.calciumImaging[ind] == 1) and (len(filenameTIF_analysis)==1):
            info.recordingList.loc[ind,'dffExtraction'] = 1
        elif (info.recordingList.calciumImaging[ind] == 1) and (len(filenameTIF_analysis)<1):# calculate dff if not calculated already
            # get PAQ data first
            print(str(ind)+ ' Creating: ' + info.recordingList.analysispathname[ind])
            filenamePAQ  = [f for f in glob.glob(info.recordingList.filepathname[ind]  + # paqs_loc
                                                '*'+ format(int(info.recordingList.recordingID[ind]), '03d') + '.paq')]
            filenamePAQ_analysis = [f for f in glob.glob(savepathname + 'paq-data.pkl')]
            paq_data = paq_reader.paq_read( file_path=filenamePAQ[0], plot=True, save_path=savepathname) # we need this below
            frame_clockPAQ  = utils.paq_data(paq_data, 'prairieFrame', threshold_ttl=True, plot=False)
            imaging_data = mfun.calculateDFF (filepathname, frame_clockPAQ)
            filenameTIF_analysis = savepathname + 'imaging-data.pkl'
            if imaging_data['imagingDataQuality']==True:
                with open(filenameTIF_analysis, 'wb') as f:
                    pickle.dump(imaging_data, f)
                    info.recordingList.loc[ind,'dffExtraction'] = 1
            else:
                print('Imaging data quality is NOT good: ' + filenameTIF[0])
            info.recordingList.loc[ind,'dffExtraction'] = 0
        elif (info.recordingList.calciumImaging[ind]) == 0 : # No imaging data recorded
            info.recordingList.loc[ind,'dffExtraction'] = 0
    #except:
        #print('Calcium imaging data is crashed - most likely two channel: ' + filepathname)
        #info.recordingList.loc[ind,'dffExtraction'] = 0
# display the output
print( "DFF extraction Completed: " + 
      str(info.recordingList.dffExtraction.sum()) +"/" + str(info.recordingList.calciumImaging.sum()))
notExtractedFiles = info.recordingList[info.recordingList['dffExtraction'] != 1]
if notExtractedFiles.shape[0]==0: print('All imaging data are extracted')

DFF extraction Completed: 280.0/280


In [7]:
#Check if pupil data is calculated & extracted

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
        # Get the imaging data if imaging file is ready (suite2p output is ready)
        filepathname = (info.recordingList.filepathname[ind] +'_p-' + 
                    format(int(info.recordingList.recordingID[ind]), '03d') )
        filenamePUPIL  = [f for f in glob.glob(filepathname + "\\*.csv")] # datapath
        savepathname = info.recordingList.analysispathname[ind]
        filenamePUPIL_analysis = [f for f in glob.glob(savepathname + 'pupil-data.pkl')]
        if (info.recordingList.pupilcsvcreated[ind] == 1) and (len(filenamePUPIL_analysis)==1):
            info.recordingList.loc[ind,'pupilExtraction'] = 1
        elif (info.recordingList.pupilcsvcreated[ind] == 1) and len(filenamePUPIL_analysis)<1: # calculate dff if not calculated already
            # get PAQ data first
            print(str(ind)+ ' Creating: ' + info.recordingList.analysispathname[ind])
            filenamePAQ  = [f for f in glob.glob(info.recordingList.filepathname[ind]  + # paqs_loc
                                                '*'+ format(int(info.recordingList.recordingID[ind]), '03d') + '.paq')]
            filenamePAQ_analysis = [f for f in glob.glob(savepathname + 'paq-data.pkl')]
            paq_data = paq_reader.paq_read( file_path=filenamePAQ[0], plot=True, save_path=savepathname) # we need this below
            frame_clockPAQ  = utils.paq_data(paq_data, 'prairieFrame', threshold_ttl=True, plot=False)
            pupil_data = mfun.calculatePupil (filenamePUPIL,frame_clockPAQ)
            filenamePUPIL_analysis = savepathname + 'pupil-data.pkl'
            
            with open(filenamePUPIL_analysis, 'wb') as f:
                pickle.dump(pupil_data, f)
                info.recordingList.loc[ind,'pupilExtraction'] = 1
        else:
            info.recordingList.loc[ind,'pupilExtraction'] = 0
    #except:
     #   print('Crashed')
     #   info.recordingList.loc[ind,'pupilExtraction'] = 0

# display the output
print( "Pupil size calculation Completed: " + 
      str(info.recordingList.pupilExtraction.sum()) +"/" + str(info.recordingList.pupilImaging.sum()))
notExtractedFiles = info.recordingList[info.recordingList['pupilExtraction'] != 1]
if notExtractedFiles.shape[0]==0: print('All pupil data are extracted')

Pupil size calculation Completed: 322.0/322


In [9]:
# save info into the analysis folder
filenameINFO = info.analysisPath + '\\infoForAnalysis.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

All should be done!!
